In [2]:
pip install seaborn


Note: you may need to restart the kernel to use updated packages.


In [3]:
# ============================================================
# Notebook 01 — Global Data Cleaning (Type & Integrity Checks) (V2)
# Purpose:
# - Deterministic cleaning + integrity certification
# - DO NOT break downstream contracts (official artifacts unchanged)
# - Emit audit artifacts for traceability (schema, summary, hashes, plots)
# ============================================================

# -----------------------------
# Block 0 — Setup & Config
# -----------------------------
import os, sys, json, math, warnings, hashlib, time
from typing import Dict, List, Tuple
from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import is_numeric_dtype, is_datetime64_any_dtype

# Reproducibility & display
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 160)
warnings.filterwarnings("ignore")

RUN_ID = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")

# --- Paths (EDIT if needed)
DATA_DIR = "."   # raw CSV folder
CLEAN_DIR = "clean"  # official cleaned outputs (contract)
REPORT_DIR = os.path.join(CLEAN_DIR, "report", "nb01")

os.makedirs(CLEAN_DIR, exist_ok=True)
os.makedirs(REPORT_DIR, exist_ok=True)
os.makedirs(os.path.join(REPORT_DIR, "plots"), exist_ok=True)
os.makedirs(os.path.join(REPORT_DIR, "tables"), exist_ok=True)

# Expected file names (contracts)
FILES = {
    "sales": "sales.csv",
    "products": "products_with_category_v7_clean.csv",
    "cities": "cities.csv",
    "countries": "countries.csv",
    "customers": "customers.csv",
    "employees": "employees.csv",
}

file_paths = {k: os.path.join(DATA_DIR, v) for k, v in FILES.items()}
print("Resolved file paths:\n", json.dumps(file_paths, indent=2))

def assert_files_exist(paths: Dict[str, str]) -> None:
    missing = [name for name, p in paths.items() if not os.path.isfile(p)]
    if missing:
        raise FileNotFoundError(f"Required files not found in DATA_DIR: {missing}")

def save_json(obj: dict, path: str):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, default=str)
    print(f"[audit] Saved {path}")

def md5_of_file(path: str) -> str:
    h = hashlib.md5()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

def dataframe_schema(df: pd.DataFrame) -> List[dict]:
    return [{"column": c, "dtype": str(df[c].dtype)} for c in df.columns]

# -----------------------------
# Block 1 — Load Raw CSVs
# -----------------------------
assert_files_exist(file_paths)

dfs: Dict[str, pd.DataFrame] = {}
for name, path in file_paths.items():
    dfs[name] = pd.read_csv(path)
    print(f"[load] {name}: {dfs[name].shape[0]:,} rows × {dfs[name].shape[1]} cols")

# -----------------------------
# Block 2 — Basic Type Standards
# -----------------------------
# Dates (tz-naive)
dt_cols = {
    "sales": ["SalesDate"],
    "products": ["ModifyDate"],
    "employees": ["BirthDate", "HireDate"],
}
for tname, cols in dt_cols.items():
    for c in cols:
        if c in dfs[tname].columns:
            dfs[tname][c] = pd.to_datetime(dfs[tname][c], errors="coerce")

# City / Country names
if "CityName" in dfs["cities"].columns:
    dfs["cities"]["CityName"] = dfs["cities"]["CityName"].astype(str).str.strip().str.title()
if "CountryName" in dfs["countries"].columns:
    dfs["countries"]["CountryName"] = dfs["countries"]["CountryName"].astype(str).str.strip().str.title()

# Product categorical-like columns
cat_cols_products = ["Category", "Class", "Resistant", "IsAllergic", "ShelfLifeStatus"]
for col in cat_cols_products:
    if col in dfs["products"].columns:
        dfs["products"][col] = dfs["products"][col].astype(str).str.strip().str.title()

# Employees gender standardization
if "Gender" in dfs["employees"].columns:
    dfs["employees"]["Gender"] = (
        dfs["employees"]["Gender"].astype(str).str.strip().str.upper().replace({"FEMALE": "F", "MALE": "M"})
    )

# ID casting (non-failing)
id_cols = {"SalesID","SalesPersonID","CustomerID","ProductID","CategoryID","CityID","CountryID","EmployeeID"}
for name, df in dfs.items():
    for col in df.columns:
        if col in id_cols:
            try:
                dfs[name][col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
            except Exception:
                pass

# Quick numeric sanity (products Price)
if "Price" in dfs["products"].columns:
    negative_prices = dfs["products"].loc[dfs["products"]["Price"].fillna(0) <= 0]
    if len(negative_prices):
        print("⚠️ Found products with non-positive Price. Sample:")
        print(negative_prices[["ProductID","ProductName","Price"]].head(5))

# -----------------------------
# Block 3 — SALES integrity
# -----------------------------
sales = dfs["sales"].copy()

# TransactionNumber uniqueness (if present)
if "TransactionNumber" in sales.columns:
    print(f"[sales] TransactionNumber unique? {sales['TransactionNumber'].is_unique}")

# Basic anomaly snapshots
zero_price_records = sales[sales.get("TotalPrice", pd.Series(dtype=float)).fillna(0) == 0.0]
null_salesdate_records = sales[sales["SalesDate"].isna()]
print(f"[sales] TotalPrice = 0.0 rows: {len(zero_price_records):,}")
print(f"[sales] Null SalesDate rows: {len(null_salesdate_records):,}")

# Flags
sales["MissingSalesDate"] = sales["SalesDate"].isna()
sales["Note_TotalPrice"] = "Needs recalculation from products"
dfs["sales"] = sales

# -----------------------------
# Block 4 — PRODUCTS integrity
# -----------------------------
products = dfs["products"].copy()

if "VitalityDays" in products.columns:
    vitality_zero = products[products["VitalityDays"] == 0.0]
    print(f"[products] VitalityDays == 0.0: {len(vitality_zero):,}")

# PK & duplicates
if products["ProductID"].duplicated().any():
    print("⚠️ Duplicate ProductID detected — de-duplicating by ProductID.")
    products = products.drop_duplicates(subset=["ProductID"])

products = products.drop_duplicates()

# Binary fields normalization if exist
for bcol in ["IsFoodProduct", "IsAllergic_bin"]:
    if bcol in products.columns:
        try:
            if bcol == "IsAllergic_bin":
                products[bcol] = pd.to_numeric(products[bcol], errors="coerce").round().astype("Int64")
            else:
                products[bcol] = pd.to_numeric(products[bcol], errors="coerce").fillna(0).astype("Int64")
        except Exception:
            pass

# Perishable flag from VitalityDays (>0 → perishable)
if "VitalityDays" in products.columns:
    products["Perishable"] = (products["VitalityDays"].fillna(0) > 0).astype("Int64")

dfs["products"] = products

# -----------------------------
# Block 5 — CITIES integrity
# -----------------------------
cities = dfs["cities"].copy()
if "CityName" in cities.columns:
    cities["CityName"] = cities["CityName"].astype(str).str.strip().str.title()

# Zipcode checks
if "Zipcode" in cities.columns:
    cities["Zipcode_str"] = cities["Zipcode"].astype(str)
    invalid_zip = cities[~cities["Zipcode_str"].str.match(r"^\d{4,6}$")]
    print(f"[cities] Invalid Zipcodes: {len(invalid_zip)}")
    cities["ZipcodeValid"] = cities["Zipcode_str"].str.match(r"^\d{4,6}$")
    cities = cities.drop(columns=["Zipcode_str"])
dfs["cities"] = cities

# -----------------------------
# Block 6 — COUNTRIES integrity
# -----------------------------
countries = dfs["countries"].copy()
if "CountryName" in countries.columns:
    countries["CountryName"] = countries["CountryName"].astype(str).str.strip().str.title()
if "CountryCode" in countries.columns:
    invalid_codes = countries[~countries["CountryCode"].astype(str).str.match(r"^[A-Za-z]{2,3}$", na=True)]
    print(f"[countries] Invalid CountryCode format: {len(invalid_codes)}")
    countries["CountryCode"] = countries["CountryCode"].fillna("UNK")
dfs["countries"] = countries

# -----------------------------
# Block 7 — CUSTOMERS integrity & FK
# -----------------------------
customers = dfs["customers"].copy()
def _clean_text_series(s: pd.Series) -> pd.Series:
    return (s.astype(str).str.strip().str.replace(r"\s+", " ", regex=True).str.title())

for c in ["FirstName","LastName","Address"]:
    if c in customers.columns: customers[c] = _clean_text_series(customers[c])

if "MiddleInitial" in customers.columns:
    customers["MiddleInitial_missing"] = customers["MiddleInitial"].isna()
    customers["MiddleInitial"] = customers["MiddleInitial"].fillna("").astype(str).str.strip()

# PK
if "CustomerID" in customers.columns:
    dup = customers["CustomerID"].duplicated().sum()
    print(f"[customers] Duplicate CustomerID rows: {dup}")

# FK City
if "CityID" in customers.columns:
    valid_city_ids = set(dfs["cities"]["CityID"].dropna().astype(int).unique())
    customers["CityID_valid"] = customers["CityID"].astype("Int64").isin(valid_city_ids)

# Drop exact duplicates
customers = customers.drop_duplicates()
dfs["customers"] = customers

# -----------------------------
# Block 8 — EMPLOYEES integrity & FK
# -----------------------------
employees = dfs["employees"].copy()
for c in ["FirstName","LastName"]:
    if c in employees.columns: employees[c] = _clean_text_series(employees[c])

if "EmployeeID" in employees.columns:
    dup_emp = employees["EmployeeID"].duplicated().sum()
    print(f"[employees] Duplicate EmployeeID rows: {dup_emp}")

if "CityID" in employees.columns:
    valid_city_ids = set(dfs["cities"]["CityID"].dropna().astype(int).unique())
    employees["CityID_valid"] = employees["CityID"].astype("Int64").isin(valid_city_ids)

employees = employees.drop_duplicates()
dfs["employees"] = employees

# -----------------------------
# Block 9 — Global FK checks
# -----------------------------
def check_fk(parent_df, child_df, parent_key, child_key) -> set:
    parent_keys = set(parent_df[parent_key].dropna().astype(int).unique())
    child_keys  = set(child_df[child_key].dropna().astype(int).unique())
    return child_keys - parent_keys

fk_report = {}

if {"CustomerID"} <= set(dfs["sales"].columns):
    miss = check_fk(dfs["customers"], dfs["sales"], "CustomerID", "CustomerID")
    fk_report["sales->customers"] = len(miss)

if {"ProductID"} <= set(dfs["sales"].columns):
    miss = check_fk(dfs["products"], dfs["sales"], "ProductID", "ProductID")
    fk_report["sales->products"] = len(miss)

if {"SalesPersonID"} <= set(dfs["sales"].columns):
    miss = check_fk(dfs["employees"], dfs["sales"], "EmployeeID", "SalesPersonID")
    fk_report["sales->employees"] = len(miss)

if "CityID" in dfs["customers"].columns:
    miss = check_fk(dfs["cities"], dfs["customers"], "CityID", "CityID")
    fk_report["customers->cities"] = len(miss)

if "CityID" in dfs["employees"].columns:
    miss = check_fk(dfs["cities"], dfs["employees"], "CityID", "CityID")
    fk_report["employees->cities"] = len(miss)

if "CountryID" in dfs["cities"].columns:
    miss = check_fk(dfs["countries"], dfs["cities"], "CountryID", "CountryID")
    fk_report["cities->countries"] = len(miss)

print("[fk] Missing key counts:", fk_report)

# -----------------------------
# Block 10 — Data Dictionary (auto)
# -----------------------------
def summarize_column(s: pd.Series) -> dict:
    total = len(s)
    nulls = int(s.isna().sum())
    non_null = int(total - nulls)
    dtype = str(s.dtype)
    try:
        n_unique = int(s.nunique(dropna=True))
    except Exception:
        n_unique = None
    min_val = max_val = None
    try:
        if is_numeric_dtype(s) or is_datetime64_any_dtype(s):
            min_val, max_val = s.min(), s.max()
    except Exception:
        pass
    try:
        samples = s.dropna().astype(str).unique()[:3].tolist()
    except Exception:
        samples = []
    return {
        "dtype": dtype, "non_null": non_null, "nulls": nulls,
        "null_pct": round((nulls/total)*100, 4) if total else 0.0,
        "n_unique": n_unique, "min": min_val, "max": max_val,
        "sample_values": samples
    }

rows = []
for tname, df in dfs.items():
    for col in df.columns:
        info = summarize_column(df[col])
        rows.append({"table": tname, "column": col, **info})

data_dict = pd.DataFrame(rows).sort_values(["table","column"]).reset_index(drop=True)

key_notes = {
    ("sales", "SalesID"): "PK",
    ("sales", "CustomerID"): "FK -> customers.CustomerID",
    ("sales", "ProductID"): "FK -> products.ProductID",
    ("sales", "SalesPersonID"): "FK -> employees.EmployeeID",
    ("customers", "CustomerID"): "PK",
    ("customers", "CityID"): "FK -> cities.CityID",
    ("employees", "EmployeeID"): "PK",
    ("employees", "CityID"): "FK -> cities.CityID",
    ("cities", "CityID"): "PK",
    ("cities", "CountryID"): "FK -> countries.CountryID",
    ("countries", "CountryID"): "PK",
    ("products", "ProductID"): "PK",
    ("products", "CategoryID"): "(dim attribute)",
}
data_dict["key_note"] = data_dict.apply(lambda r: key_notes.get((r["table"], r["column"]), ""), axis=1)

dict_path = os.path.join(REPORT_DIR, "tables", "data_dictionary_nb01.csv")
data_dict.to_csv(dict_path, index=False)
print(f"[audit] Data dictionary → {dict_path}")

# -----------------------------
# Block 11 — Pre-merge: recompute TotalPrice (sales × products)
# -----------------------------
sales = dfs["sales"].copy()
prod_merge_cols = [c for c in ["ProductID","Price","Perishable","Category","Class"] if c in dfs["products"].columns]
products = dfs["products"][prod_merge_cols].copy()

merged = sales.merge(
    products, on="ProductID", how="left", validate="many_to_one", indicator=True
)
merged["MissingProduct"] = merged["_merge"].ne("both")
merged = merged.drop(columns="_merge")

# Normalize Discount
if "Discount" in merged.columns:
    merged["Discount"] = pd.to_numeric(merged["Discount"], errors="coerce").fillna(0.0).clip(0.0, 1.0)
else:
    merged["Discount"] = 0.0

# Compute totals
if "Price" in merged.columns and "Quantity" in merged.columns:
    merged["UnitNetPrice"] = merged["Price"] * (1.0 - merged["Discount"])
    merged["ComputedTotalPrice"] = (pd.to_numeric(merged["Quantity"], errors="coerce") * merged["UnitNetPrice"]).astype(float)
else:
    merged["UnitNetPrice"] = np.nan
    merged["ComputedTotalPrice"] = np.nan

merged["ComputedTotalPriceMissing"] = merged["ComputedTotalPrice"].isna()
if "TotalPrice" in merged.columns:
    merged = merged.rename(columns={"TotalPrice": "TotalPrice_old"})
merged["TotalPrice"] = merged["ComputedTotalPrice"].round(2)

print(f"[merge] Rows: {len(merged):,} | Missing product info: {int(merged['MissingProduct'].sum()):,} | Missing computed total: {int(merged['ComputedTotalPriceMissing'].sum()):,}")

# Save enriched sales (official location unchanged from your previous code)
enriched_path = os.path.join(CLEAN_DIR, "sales_enriched.parquet")
merged.to_parquet(enriched_path, index=False)
print(f"[write] Enriched sales → {enriched_path}")

dfs["sales_enriched"] = merged

# -----------------------------
# Block 12 — Continuous Calendar & Missing-Day Flags (sales daily)
# -----------------------------
# Build a daily calendar and flag missing days at aggregate level (useful for forecasting readiness)
if "SalesDate" in sales.columns:
    min_d, max_d = sales["SalesDate"].min(), sales["SalesDate"].max()
    if pd.notna(min_d) and pd.notna(max_d):
        calendar = pd.DataFrame({"SalesDate": pd.date_range(min_d.normalize(), max_d.normalize(), freq="D")})
        sales_daily = (
            sales.assign(TotalPrice=sales.get("TotalPrice", pd.Series(dtype=float)))
                 .groupby(pd.Grouper(key="SalesDate", freq="D"), dropna=False)["SalesID"]
                 .count()
                 .rename("daily_txn_count").reset_index()
        )
        sales_daily_full = calendar.merge(sales_daily, on="SalesDate", how="left")
        sales_daily_full["is_missing_day"] = sales_daily_full["daily_txn_count"].isna()
        # Save to audit tables
        cal_out = os.path.join(REPORT_DIR, "tables", "sales_daily_calendar.csv")
        sales_daily_full.to_csv(cal_out, index=False)
        print(f"[audit] Sales daily calendar → {cal_out}")
    else:
        print("[calendar] SalesDate min/max not available; skipping calendar build.")

# -----------------------------
# Block 13 — Audit Plots (missingness + outliers)
# -----------------------------
def plot_missing_matrix(df: pd.DataFrame, title: str, out_path: str, max_rows: int = 300):
    sample = df.head(max_rows).copy()
    miss = sample.isna().astype(int)
    plt.figure(figsize=(min(18, 0.25*miss.shape[1] + 6), 6))
    sns.heatmap(miss.T, cbar=False)
    plt.title(title)
    plt.xlabel("Row index (sample)")
    plt.ylabel("Columns")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()
    print(f"[plot] {out_path}")

def plot_box_outliers(df: pd.DataFrame, title: str, out_path: str, cols: List[str]):
    num_cols = [c for c in cols if c in df.columns and is_numeric_dtype(df[c])]
    if not num_cols:
        print(f"[plot] No numeric columns found for outlier boxplot: {title}")
        return
    plt.figure(figsize=(min(18, 2*len(num_cols)+6), 6))
    sns.boxplot(data=df[num_cols], orient="h")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()
    print(f"[plot] {out_path}")

# Missing matrix for key tables
plot_missing_matrix(dfs["sales"], "Missingness Matrix — sales (head 300)", os.path.join(REPORT_DIR, "plots", "missing_matrix_sales.png"))
plot_missing_matrix(dfs["products"], "Missingness Matrix — products (head 300)", os.path.join(REPORT_DIR, "plots", "missing_matrix_products.png"))

# Outliers boxplot for common numeric fields
box_cols_sales = [c for c in ["Quantity","Discount","TotalPrice","ComputedTotalPrice","UnitNetPrice"] if c in merged.columns]
plot_box_outliers(merged, "Outliers — sales_enriched (key numeric)", os.path.join(REPORT_DIR, "plots", "outliers_box_sales_enriched.png"), box_cols_sales)

box_cols_products = [c for c in ["Price","VitalityDays"] if c in products.columns]
plot_box_outliers(products, "Outliers — products (Price/VitalityDays)", os.path.join(REPORT_DIR, "plots", "outliers_box_products.png"), box_cols_products)

# -----------------------------
# Block 14 — Write Official Clean Tables (CONTRACTS)
# -----------------------------
# Keep names as current practice; downstream notebooks rely on these.
for name, df in dfs.items():
    out_path = os.path.join(CLEAN_DIR, f"{name}.parquet")
    df.to_parquet(out_path, index=False)
    print(f"[write] {name} → {out_path} ({len(df):,} rows)")

print("\n✅ All cleaned tables saved to 'clean/'.")

# -----------------------------
# Block 15 — Schema & Summary (audit)
# -----------------------------
schema_payload = {
    "run_id": RUN_ID,
    "random_seed": RANDOM_SEED,
    "tables": {name: dataframe_schema(df) for name, df in dfs.items()},
}
save_json(schema_payload, os.path.join(REPORT_DIR, "schema.json"))

summary_payload = {
    "run_id": RUN_ID,
    "timestamp_utc": datetime.utcnow().isoformat(),
    "row_counts": {name: int(len(df)) for name, df in dfs.items()},
    "missing_pct_by_table": {
        name: round(float(df.isna().sum().sum() / (df.shape[0]*max(df.shape[1],1)) * 100), 4) if len(df) and df.shape[1] else 0.0
        for name, df in dfs.items()
    },
    "sales_date_range": {
        "min": str(dfs["sales"]["SalesDate"].min()) if "SalesDate" in dfs["sales"].columns else None,
        "max": str(dfs["sales"]["SalesDate"].max()) if "SalesDate" in dfs["sales"].columns else None,
    },
    "fk_missing_counts": fk_report,
}
save_json(summary_payload, os.path.join(REPORT_DIR, "summary.json"))

# -----------------------------
# Block 16 — Hashes of Official Artifacts
# -----------------------------
hashes = {}
for fname in os.listdir(CLEAN_DIR):
    if fname.endswith(".parquet"):
        fpath = os.path.join(CLEAN_DIR, fname)
        hashes[fname] = md5_of_file(fpath)

hashes_payload = {
    "run_id": RUN_ID,
    "artifacts_md5": hashes
}
save_json(hashes_payload, os.path.join(REPORT_DIR, "hashes.json"))

print("\n🎯 Notebook 01 (V2) finished with audit artifacts at:", REPORT_DIR)


Resolved file paths:
 {
  "sales": "./sales.csv",
  "products": "./products_with_category_v7_clean.csv",
  "cities": "./cities.csv",
  "countries": "./countries.csv",
  "customers": "./customers.csv",
  "employees": "./employees.csv"
}
[load] sales: 6,758,125 rows × 9 cols
[load] products: 452 rows × 14 cols
[load] cities: 96 rows × 4 cols
[load] countries: 206 rows × 3 cols
[load] customers: 98,759 rows × 6 cols
[load] employees: 23 rows × 8 cols
[sales] TransactionNumber unique? True
[sales] TotalPrice = 0.0 rows: 6,758,125
[sales] Null SalesDate rows: 67,526
[products] VitalityDays == 0.0: 43
[cities] Invalid Zipcodes: 3
[countries] Invalid CountryCode format: 0
[customers] Duplicate CustomerID rows: 0
[employees] Duplicate EmployeeID rows: 0
[fk] Missing key counts: {'sales->customers': 0, 'sales->products': 0, 'sales->employees': 0, 'customers->cities': 0, 'employees->cities': 0, 'cities->countries': 0}
[audit] Data dictionary → clean/report/nb01/tables/data_dictionary_nb01.csv
[m